In [7]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

selecoes= pd.read_excel('./DadosCopaDoMundoQatar2022.xlsx',sheet_name='selecoes', index_col = 0)
jogos = pd.read_excel('./DadosCopaDoMundoQatar2022.xlsx', sheet_name ='jogos')

In [8]:
selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [9]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [10]:
fifa= selecoes['PontosRankingFIFA']
fifa

Seleção
Catar             1442
Equador           1464
Senegal           1585
Holanda           1679
Inglaterra        1737
Irã               1559
Estados Unidos    1635
País de Gales     1582
Argentina         1771
Arábia Saudita    1436
México            1650
Polônia           1546
França            1765
Dinamarca         1665
Tunísia           1508
Austrália         1484
Espanha           1717
Alemanha          1659
Japão             1555
Costa Rica        1500
Bélgica           1822
Canadá            1474
Marrocos          1558
Croácia           1632
Brasil            1838
Sérvia            1550
Suíça             1621
Camarões          1485
Portugal          1679
Gana              1393
Uruguai           1641
Coreia do Sul     1526
Name: PontosRankingFIFA, dtype: int64

In [11]:
a, b= min(fifa), max(fifa)
fa, fb = 0.15, 1
b1= (fb-fa)/(b-a)
b0= fb - b*b1
forca= b0 + b1*fifa

forca.sort_values(ascending= False)

Seleção
Brasil            1.000000
Bélgica           0.969438
Argentina         0.872022
França            0.860562
Inglaterra        0.807079
Espanha           0.768876
Holanda           0.696292
Portugal          0.696292
Dinamarca         0.669551
Alemanha          0.658090
México            0.640899
Uruguai           0.623708
Estados Unidos    0.612247
Croácia           0.606517
Suíça             0.585506
Senegal           0.516742
País de Gales     0.511011
Irã               0.467079
Marrocos          0.465169
Japão             0.459438
Sérvia            0.449888
Polônia           0.442247
Coreia do Sul     0.404045
Tunísia           0.369663
Costa Rica        0.354382
Camarões          0.325730
Austrália         0.323820
Canadá            0.304719
Equador           0.285618
Catar             0.243596
Arábia Saudita    0.232135
Gana              0.150000
Name: PontosRankingFIFA, dtype: float64

In [18]:
def Resultado(gols1, gols2):
    if(gols1>gols2):
        res= 'V'
    if(gols1<gols2):
        res= 'D'
    if(gols1==gols2):
        res='E'
    return res

def MediasPoisson(sel1, sel2):
    forca1= forca[sel1]
    forca2= forca[sel2]

    mgols= 2.75
    l1= mgols*forca1 / (forca1+forca2)
    l2= mgols*forca2 / (forca1+forca2)
    
    return [l1,l2]

def Distribuicao(media):
    probs= []
    for i in range(7):
        probs.append(poisson.pmf(i, media))
    probs.append(1 - sum(probs))
    return pd.Series(probs, index=['0', '1', '2', '3', '4', '5', '6', '7+'])

def ProbabilidadesPartida(selecao1, selecao2):
    l1, l2= MediasPoisson(selecao1, selecao2)
    d1, d2= Distribuicao(l1), Distribuicao(l2)
    matriz= np.outer(d1, d2)

    vitoria= np.tril(matriz).sum()- np.trace(matriz)
    derrota= np.triu(matriz).sum()- np.trace(matriz)
    probs= np.around([vitoria, 1-(vitoria+derrota), derrota], 3)
    probsp= [f'{100*i:.1f}%' for i in probs]

    names= ['0', '1', '2', '3', '4', '5', '6', '7+']
    matriz= pd.DataFrame(matriz, columns=names, index= names)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns])

    output= {"seleção 1": selecao1, 'seleção2': selecao2, 'f1': forca[selecao1], 'f2': forca[selecao2], 'media1': l1, 'media2': l2, 'probs': probsp, 'matriz': matriz}

    return output

def Pontos(gols1, gols2):
    result = Resultado(gols1, gols2)
    if result== 'V':
        pontos1, pontos2= 3, 0
    if result== 'E':
        pontos1, pontos2= 1,1
    if result== 'D':
        pontos1, pontos2= 0, 3

    return pontos1, pontos2, result

def Jogo(selecao1, selecao2):
    l1, l2= MediasPoisson(selecao1, selecao2)
    gols1= int(np.random.poisson(lam= l1, size=1))
    gols2= int(np.random.poisson(lam= l2, size=1))
    
    saldo1= gols1 - gols2
    saldo2= -saldo1
    pontos1, pontos2, result= Pontos(gols1, gols2)
    placar= f'{gols1}X{gols2}'

    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, result, placar]


In [ ]:
def JogoMM(selecao1, selecao2):
    jogo= Jogo(selecao1, selecao2)
    